# Python examples of Control Plane bindings

*This notebook is developed and test on the FABRIC sender node.*

First import the library "e2sar_py". Make sure the module is included in the path.

In [1]:
import sys

## IMPORTANT: Update the path to your built Python module
sys.path.append(
    '/home/ubuntu/dev-e2sar/build/src/pybind')

import e2sar_py
# print(e2sar_py.get_version())

In [2]:
# print(dir(e2sar_py.ControlPlane))

## Helper classes

#### "Timestamp" class

The Python `e2sar_py.ControlPlane.Timestamp` class serves as a bridge to the C++ `google::protobuf::Timestamp` class. The following code illustrates how to use it. Note that the object is printed as a string.

In [3]:
%pip install protobuf

'''Cannot direct map Python protobuf TimeStamp to C++ protobuf TimeStamp So we created our own.'''

from google.protobuf.timestamp_pb2 import Timestamp as gts
from e2sar_py.ControlPlane import Timestamp


def get_currtimestamp_from_gts() -> e2sar_py.ControlPlane.Timestamp:
    g_ts = gts()
    g_ts.GetCurrentTime()
    curr_ts = Timestamp()
    curr_ts.set_seconds(g_ts.seconds)
    curr_ts.set_nanos(g_ts.nanos)
    return curr_ts

ts = get_currtimestamp_from_gts()
print(f"Timestamp: {ts}, seconds = {ts.get_seconds()}, nanos = {ts.get_nanos()}")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Timestamp: 2024-10-02T04:14:25.698545Z, seconds = 1727842465, nanos = 698545000


#### "LBWorkerStatus" class

The following code block demonstrates how to manipulate the `ControlPlane.LBWorkerStatus` class. The `last_updated` attribute is a string.

In [4]:
worker = e2sar_py.ControlPlane.LBWorkerStatus("Worker1", 75.5, 0.9, 10, ts)

print(f"Worker name: {worker.name}")
print(f"Worker fill percent: {worker.fill_percent}")
print(f"Worker control signal: {worker.control_signal}")
print(f"Worker slots assigned: {worker.slots_assigned}")
print(f"Worker last updated: {worker.last_updated}")

assert(worker.last_updated == str(ts))

Worker name: Worker1
Worker fill percent: 75.5
Worker control signal: 0.8999999761581421
Worker slots assigned: 10
Worker last updated: 2024-10-02T04:14:25.698545Z


#### "LBStatus" class
The attributes `timestamp` and `expiresAt` are bound as Python strings.

In [5]:
ip_list = ["192.168.100.1", "192.168.100.2"]

# Set the expire timestamp
expire_ts = Timestamp()  # DO NOT decalre as "expire_ts = ts"
expire_ts.set_seconds(ts.get_seconds() + 3600)  # 1 hr
expire_ts.set_nanos(ts.get_nanos()) 

# Create an LBStatus object with empty WorkerStatus list
status = e2sar_py.ControlPlane.LBStatus(ts, 1234, 5678, [], ip_list, expire_ts)

# Access members
assert(status.timestamp == str(ts))
assert(status.currentEpoch == 1234)
assert(status.currentPredictedEventNumber == 5678)
assert(status.senderAddressList == ip_list)
assert(status.workerStatusList == [])
assert(status.expiresAt == str(expire_ts))

print("Timestamp: ", status.timestamp)
print("Expires at: ", status.expiresAt)

Timestamp:  2024-10-02T04:14:25.698545Z
Expires at:  2024-10-02T05:14:25.698545Z


#### "OverviewEntry" class

The attributes of this class are read-only. Its usage will be demonstrated in `LBMLiveTest6`.

## "LBManager" class

In [6]:
# print(dir(e2sar_py.ControlPlane.LBManager))

### Examples with a mock EJFAT Load Balancer

This section requires a functional (mock) load balancer. If you are launching with a FABRIC slice, ensure the `udplbd` container is running on the FABRIC `cpnode` before executing the code blocks.

The Python code blocks below replicate the tests performed in the C++ `e2sar_lbcp_live_test`.

In [7]:
# Verify env variable EJFAT_URI is set
import os

# This one works on the FABRIC sender node
FABRIC_CP_URI="ejfats://udplbd@192.168.0.3:18347/lb/1?data=127.0.0.1&sync=192.168.88.199:1234"

os.environ['EJFAT_URI'] = FABRIC_CP_URI

print(f"EJFAT_URI={os.environ['EJFAT_URI']}")

EJFAT_URI=ejfats://udplbd@192.168.0.3:18347/lb/1?data=127.0.0.1&sync=192.168.88.199:1234


In [8]:
# Create an EjfatURI object
uri_rt_obj = e2sar_py.EjfatURI.get_from_env()
assert(uri_rt_obj.has_error() == False)  # make sure the EjfatURI object is created successfully

#### LBMLiveTest1

Create a `LBManager` object, reserve the LB for 1 hour, and then free the LB.

In [9]:
# Create a LBManager object
lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)
print(lbm.get_uri())  # Make sure the uri agrees with the env var

ejfats://udplbd@192.168.0.3:18347/lb/1?sync=192.168.88.199:1234&data=127.0.0.1


**Reserve a LB with duration in seconds**

The Python `datetime.timedelta` package is very convenient to get durations. For simplicity, we create a C++ method `LBManager::reserveLB` based on duration in seconds.

In [10]:
from datetime import timedelta

# Reserve for 1 hour
duration_seconds = timedelta(hours=1).total_seconds()
ip_list = ["192.168.100.1", "192.168.100.2"]

try:
    res = lbm.reserve_lb_in_seconds(
        lb_id="ejfat", seconds=duration_seconds, senders=ip_list)
    if res.has_error():
        print("Reserve LB by seconds error: ", res.error())
    else:
        print("Create LBManager obj succeeded: ", res.value())  # the n-th active LBManager
except RuntimeError as e:
    print("Caught runtime error:", e)

Create LBManager obj succeeded:  0


If you see a "failed to reserve load balancer: a maximum of 4 load balancers are supported" error, restart the `udplbd` container on the FABRIC `cpnode` and try again.

In [11]:
# Some basic verification
rt_uri = lbm.get_uri()
assert rt_uri.has_sync_addr() == True
assert rt_uri.has_data_addr() == True
assert rt_uri.get_instance_token().value() != ""
print(f"Instance token: {rt_uri.get_instance_token().value()}")

Instance token: 955d427b68d84c56e0f197dbcb80ff5217b21c7ff13e5d4334ec33dd31fe177f


In [12]:
# Free LB
res = lbm.free_lb()
if res.has_error():
    print("Free LB error: ", res.error())
assert res.value() == 0

#### LBMLiveTest2

In [13]:
# Create the LBManager object and reserve for 1 hour
lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)

try:
    res = lbm.reserve_lb_in_seconds(
        lb_id="ejfat", seconds=duration_seconds, senders=ip_list)
    if res.has_error():
        print("Reserve LB by seconds error: ", res.error())
    else:
        print("Create LBManager obj succeeded: ", res.value())
except RuntimeError as e:
    print("Caught runtime error:", e)

Create LBManager obj succeeded:  0


In [14]:
sync_addr_1 = lbm.get_uri().get_sync_addr().value()
print("Sync IPv4 addr is: ", sync_addr_1[0])
lb_id = lbm.get_uri().lb_id
print("LB id is: ", lb_id)

Sync IPv4 addr is:  192.168.0.3
LB id is:  2


In [15]:
# Creat a new EjfatURI obj from env var
new_uri_rt = e2sar_py.EjfatURI.get_from_env()
if new_uri_rt.has_error():
    print("Return EjfatURI from env var error: ", new_uri_rt.error())
print(new_uri_rt.value())

ejfats://udplbd@192.168.0.3:18347/lb/1?sync=192.168.88.199:1234&data=127.0.0.1


In [16]:
# Create another LBManager object based on the new EjfatURI obj
lbm_new = e2sar_py.ControlPlane.LBManager(new_uri_rt.value(), False)

# LB ID is an integer. Make sure to convert it to a string
res = lbm_new.get_lb_by_id(str(lb_id))
if res.has_error():
    print("Get LB by id error: ", res.error())
assert res.has_error() == False

In [17]:
# Ensure the 2 LBManager objects have the same sync address
sync_addr_2 = lbm_new.get_uri().get_sync_addr().value()
print("New sync addr is: ", sync_addr_2[0])
assert str(sync_addr_1[0]) == str(sync_addr_2[0])
assert sync_addr_1[1] == sync_addr_2[1]

New sync addr is:  192.168.0.3


In [18]:
def free_lbmanager(lb_obj : e2sar_py.ControlPlane.LBManager):
    res = lb_obj.free_lb()
    if res.has_error():
        print(f"Free LB error: ", res.error())
    assert res.value() == 0
    print("Free LB succeeded!")

free_lbmanager(lbm)

Free LB succeeded!


#### LBMLiveTest2_1

Create a `LBManager` object with empty senders list. Add and remove the senders later.

In [19]:
# Create the LBManager object from an empty sender list
lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)

try:
    res = lbm.reserve_lb_in_seconds(
        lb_id="ejfat", seconds=duration_seconds, senders=[])
    if res.has_error():
        print("Reserve LB by seconds error: ", res.error())
        free_lbmanager(lbm)
    else:
        print("Create LBManager obj succeeded: ", res.value())
except RuntimeError as e:
    print("Caught runtime error:", e)

assert lbm.get_uri().has_sync_addr() == True
assert lbm.get_uri().has_data_addr() == True

Create LBManager obj succeeded:  0


In [20]:
# Add senders list
res = lbm.add_senders(ip_list)
if res.has_error():
    print("Add senders to LBManager error: ", res.error())
    free_lbmanager(lbm)

In [21]:
# Create a new LBManager object
lb_id = lbm.get_uri().lb_id
print("LB ID is: ", lb_id)

lbm_new = e2sar_py.ControlPlane.LBManager(new_uri_rt.value(), False)

# LB ID is an integer. Make sure to convert it to a string
res = lbm_new.get_lb_by_id(str(lb_id))
if res.has_error():
    print("Get LB by id error: ", res.error())
assert res.has_error() == False

LB ID is:  3


In [22]:
# Compare the 2 objs' sync addresses
sync_addr_1 = lbm.get_uri().get_sync_addr().value()
sync_addr_2 = lbm_new.get_uri().get_sync_addr().value()
assert(str(sync_addr_1[0]) == str(sync_addr_2[0]))
assert(sync_addr_1[1] == sync_addr_2[1])

In [23]:
# Remove senders from the LBManager
res = lbm.remove_senders(ip_list)
if res.has_error():
    print("Remove senders from LBManager error: ", res.error())
    free_lbmanager(lbm)
assert(res.has_error() == False)

In [24]:
free_lbmanager(lbm)

Free LB succeeded!


#### LBMLiveTest3

Register and unregister worker.

In [25]:
# Create a LBManager obj and reserve for 1 hr
lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)

try:
    res = lbm.reserve_lb_in_seconds(
        lb_id="ejfat", seconds=duration_seconds, senders=ip_list)
    if res.has_error():
        print("Reserve LB by seconds error: ", res.error())
        free_lbmanager(lbm)
    else:
        print("Create LBManager obj succeeded: ", res.value())
except RuntimeError as e:
    print("Caught runtime error:", e)

assert lbm.get_uri().has_sync_addr() == True
assert lbm.get_uri().has_data_addr() == True

Create LBManager obj succeeded:  0


Every registered worker is required to send its state every 100ms; otherwise, it will be automatically deregistered. A single code block ensures that `register_worker`, `send_state`, and `deregister_worker` all execute within the 100ms timeframe.

In [26]:
# Register worker
res = lbm.register_worker(
    "my_node",  # node name
    (e2sar_py.IPAddress.from_string("192.168.101.5"), 10000),  # Ip_addr
    0.5,  # weight
    10,  # source_count
    1.0,  # min_factor
    1.0   # max_factor
)
if res.has_error():
    print("Register worker error: ", res.error())
    free_lbmanager(lbm)
assert(res.value() == 0)
print("Register worker succeeded")

# Send state
print()
res = lbm.send_state(0.8, 1.0, True)
if res.has_error():
    print("Send state error: ", res.error())
assert(res.value() == 0)
print("Send state succeeded")

# Verifications
print()
session_id = lbm.get_uri().session_id
print("Session id is: ", session_id)
session_token = lbm.get_uri().get_session_token().value()
print("Session token is: ", session_token)

# Deregister
print()
res = lbm.deregister_worker()
if res.has_error():
    print("Deregister worker error: ", res.error())
    free_lbmanager(lbm)
assert(res.value() == 0)
print("Deregister worker succeeded")

Register worker succeeded

Send state succeeded

Session id is:  1
Session token is:  9f5dbfd36e86f520eea8a808faf79129a709f896df3a1123f8bed664c400c1d4

Deregister worker succeeded


In [27]:
free_lbmanager(lbm)

Free LB succeeded!


#### LBMLiveTest4

Examples with get LB status.

In [28]:
lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)

try:
    res = lbm.reserve_lb_in_seconds(
        lb_id="ejfat", seconds=duration_seconds, senders=ip_list)
    if res.has_error():
        print("Reserve LB by seconds error: ", res.error())
        free_lbmanager(lbm)
    else:
        print("Create LBManager obj succeeded: ", res.value())
except RuntimeError as e:
    print("Caught runtime error:", e)

assert lbm.get_uri().has_sync_addr() == True
assert lbm.get_uri().has_data_addr() == True

Create LBManager obj succeeded:  0


In [29]:
# Register a worker and send states for 25 times
res = lbm.register_worker(
    "my_node",  # node name
    (e2sar_py.IPAddress.from_string("192.168.101.5"), 10000),  # ip_addr
    0.5,  # weight
    10,  # source_count
    1.0,  # min_factor
    1.0   # max_factor
)
if res.has_error():
    print("Register worker error: ", res.error())
    free_lbmanager(lbm)
assert(res.value() == 0)
print("Register worker succeeded")

import time
print("\nSend state for 25 times")
for i in range(25):
    res = lbm.send_state(0.8, 1.0, True)
    if res.has_error():
        print(f"{i + 1}-th send state error: ", res.error())
    assert(res.value() == 0)
    # print(f"{i + 1}-th send state succeeded")
    time.sleep(0.1)  # sleep for 100ms

print()
status_res = lbm.get_lb_status()
if (status_res is None):
    print("Get LB status error")
print("Get LB status succeeded: ", status_res)

# Ensure the returned senders match the fact
res = lbm.get_sender_addresses(status_res)
assert(res == ip_list)

Register worker succeeded

Send state for 25 times

Get LB status succeeded:  <e2sar_py.ControlPlane.LoadBalancerStatusReply object at 0x7f9a0e44bab0>


In [30]:
# Get worker status
workers = lbm.get_worker_statuses(status_res)

assert len(workers) == 1

def print_workerstatus(w : e2sar_py.ControlPlane.WorkerStatus):
    print(f"Worker status:")
    print(f"    name: {w.get_name()}")
    print(f"    fill_percent: {w.get_fill_percent()}")
    print(f"    control_signal: {w.get_control_signal()}")
    print(f"    slots_assigned: {w.get_slots_assigned()}")
    print(f"    last_updated: {w.get_last_updated()}")

# print a worker's status
print_workerstatus(workers[0])

Worker status:
    name: my_node
    fill_percent: 0.800000011920929
    control_signal: 1.0
    slots_assigned: 512
    last_updated: 2024-10-02T04:18:05.888130752Z


In [31]:
assert(workers[0].get_name() == "my_node")

DELTA = 0.000001
assert(abs(workers[0].get_fill_percent() - 0.8) < DELTA)
assert(abs(workers[0].get_control_signal() - 1) < DELTA)

In [32]:
lb_status = lbm.as_lb_status(status_res)
dir(lb_status)
assert(lb_status.senderAddressList == ip_list)
assert(len(lb_status.workerStatusList) == 1)
print("Timestamp: ", lb_status.timestamp)
print("expiresAt: ", lb_status.expiresAt)
print("currentEpoch: ", lb_status.currentEpoch)
print("currentPredictedEventNumber: ", lb_status.currentPredictedEventNumber)
print("workerStatusList: ", lb_status.workerStatusList)


Timestamp:  2024-10-02T04:18:05.990769793Z
expiresAt:  2024-10-02T05:18:03Z
currentEpoch:  3
currentPredictedEventNumber:  9223372036854775808
workerStatusList:  [<e2sar_py.ControlPlane.WorkerStatus object at 0x7f9a343dde70>]


In [33]:
DELTA = 0.000001

assert(lb_status.workerStatusList[0].get_name() == "my_node")
assert(abs(lb_status.workerStatusList[0].get_fill_percent() - 0.8) < DELTA)
assert(abs(lb_status.workerStatusList[0].get_control_signal() - 1) < DELTA)

In [34]:
res = lbm.deregister_worker()
if res.has_error():
    print("Deregister worker error: ", res.error())
    free_lbmanager(lbm)
else:
    print("Deregister worker succeeded")

free_lbmanager(lbm)

Deregister worker succeeded
Free LB succeeded!


#### LBMLiveTest4_1

Reserve with empty sender list first, register workers, add senders, get status, unregister workers, get status and free the LB.

In [35]:
lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)

try:
    res = lbm.reserve_lb_in_seconds(
        lb_id="ejfat", seconds=duration_seconds, senders=[])
    if res.has_error():
        print("Reserve LB by seconds error: ", res.error())
        free_lbmanager(lbm)
    else:
        print("Create LBManager obj succeeded: ", res.value())
except RuntimeError as e:
    print("Caught runtime error:", e)

assert lbm.get_uri().has_sync_addr() == True
assert lbm.get_uri().has_data_addr() == True

res = lbm.register_worker(
    "my_node",  # node name
    (e2sar_py.IPAddress.from_string("192.168.101.5"), 10000),  # ip_addr
    0.5,  # weight
    10,  # source_count
    1.0,  # min_factor
    1.0   # max_factor
)
if res.has_error():
    print("Register worker error: ", res.error())
    free_lbmanager(lbm)
assert(res.value() == 0)
print("Register worker succeeded")

print()
session_id = lbm.get_uri().session_id
print("Session id is: ", session_id)
session_token = lbm.get_uri().get_session_token().value()
print("Session token is: ", session_token)

print("\nSend state for 25 times")
for i in range(25):
    res = lbm.send_state(0.8, 1.0, True)
    if res.has_error():
        print(f"{i + 1}-th send state error: ", res.error())
    assert(res.value() == 0)
    # print(f"{i + 1}-th send state succeeded")
    time.sleep(0.1)  # sleep for 100ms

# Add senders list
res = lbm.add_senders(ip_list)
if res.has_error():
    print("Add senders to LBManager error: ", res.error())
    free_lbmanager(lbm)
print("Add senders succeeded.")

# Get LB Status
print()
status_res = lbm.get_lb_status()
if (status_res is None):
    print("Get LB status error")
print("Get LB status succeeded: ", status_res)

res = lbm.get_sender_addresses(status_res)
assert(res == ip_list)
workers = lbm.get_worker_statuses(status_res)
assert len(workers) == 1
assert(workers[0].get_name() == "my_node")
assert(abs(workers[0].get_fill_percent() - 0.8) < DELTA)
assert(abs(workers[0].get_control_signal() - 1) < DELTA)

# Remove senders
res = lbm.remove_senders(ip_list)
if res.has_error():
    print("Remove senders from LBManager error: ", res.error())
    free_lbmanager(lbm)
print("Remove senders succeeded.")

print()
status_res = lbm.get_lb_status()
if (status_res is None):
    print("Get LB status error")
print("Get LB status succeeded: ", status_res)
res = lbm.get_sender_addresses(status_res)
assert(res == [])

print()
res = lbm.deregister_worker()
if res.has_error():
    print("Deregister worker error: ", res.error())
    free_lbmanager(lbm)
else:
    print("Deregister worker succeeded")

free_lbmanager(lbm)


Create LBManager obj succeeded:  0
Register worker succeeded

Session id is:  3
Session token is:  68cb8d9db7a4b367d4d57572a785c1d68aca22c3331fa52d4fdd1e2304c5c9ce

Send state for 25 times
Add senders succeeded.

Get LB status succeeded:  <e2sar_py.ControlPlane.LoadBalancerStatusReply object at 0x7f9a343ce9b0>
Remove senders succeeded.

Get LB status succeeded:  <e2sar_py.ControlPlane.LoadBalancerStatusReply object at 0x7f9a0e5ee1f0>

Deregister worker succeeded
Free LB succeeded!


#### LBMLiveTest5
Get version tuple.

In [36]:
lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)
print("LBManager version: ", lbm.get_version())

LBManager version:  ('ef70f7fb3a537eebe83132fc5d11f44512c8691e', 'v0.3.1', 'v0.3.0')


Do not need to free the LB since no reservation is involved.

#### LBMLiveTest6

Demonstrate the usage of getting the overview of the LB.

In [37]:
ip_list = ["192.168.20.1", "192.168.20.2"]

lbm = e2sar_py.ControlPlane.LBManager(uri_rt_obj.value(), False)

try:
    res = lbm.reserve_lb_in_seconds(
        lb_id="mylb", seconds=duration_seconds, senders=ip_list)
    if res.has_error():
        print("Reserve LB by seconds error: ", res.error())
        free_lbmanager(lbm)
    else:
        print("Create LBManager obj succeeded: ", res.value())
except RuntimeError as e:
    print("Caught runtime error:", e)

assert lbm.get_uri().has_sync_addr() == True
assert lbm.get_uri().has_data_addr() == True

res = lbm.register_worker(
    "my_node",  # node name
    (e2sar_py.IPAddress.from_string("192.168.101.5"), 10000),  # ip_addr
    0.5,  # weight
    10,  # source_count
    1.0,  # min_factor
    1.0   # max_factor
)
if res.has_error():
    print("Register worker error: ", res.error())
    free_lbmanager(lbm)
assert(res.value() == 0)
print("Register worker succeeded")

print()
session_id = lbm.get_uri().session_id
print("Session id is: ", session_id)
session_token = lbm.get_uri().get_session_token().value()
print("Session token is: ", session_token)

print("\nSend state for 25 times")
for i in range(25):
    res = lbm.send_state(0.8, 1.0, True)
    if res.has_error():
        print(f"{i + 1}-th send state error: ", res.error())
    assert(res.value() == 0)
    # print(f"{i + 1}-th send state succeeded")
    time.sleep(0.1)  # sleep for 100ms

# Get LB Status
print()
status_res = lbm.get_lb_status()
if (status_res is None):
    print("Get LB status error")
print("Get LB status succeeded: ", status_res)

res = lbm.get_sender_addresses(status_res)
assert(res == ip_list)
workers = lbm.get_worker_statuses(status_res)
assert len(workers) == 1

# as_lb_status() usage
res = lbm.as_lb_status(status_res)
print(f"Sender addresses: {res.senderAddressList}")
print(f"Current Epoch: {res.currentEpoch}")
print(f"Current predicted Event No: {res.currentPredictedEventNumber}")
print(f"Workers: {res.workerStatusList}")
print(f"Timestamp: {res.timestamp}")
print(f"expiresAt: {res.expiresAt}")
assert(res.senderAddressList == ip_list)
assert(len(res.workerStatusList) == 1)
assert(res.workerStatusList[0].get_name() == "my_node")
assert(abs(res.workerStatusList[0].get_fill_percent() - 0.8) < DELTA)
assert(abs(res.workerStatusList[0].get_control_signal() - 1) < DELTA)

print("\nSend state for 25 times")
for i in range(25):
    res = lbm.send_state(0.8, 1.0, True)
    if res.has_error():
        print(f"{i + 1}-th send state error: ", res.error())
    assert(res.value() == 0)
    # print(f"{i + 1}-th send state succeeded")
    time.sleep(0.1)  # sleep for 100ms

# as_overview_message() usage
print()
overview_res = lbm.get_lb_overview()
if (overview_res is None):
    print("Get LB overview error")
    free_lbmanager(lbm)

res = lbm.as_overview_message(overview_res)   # a list
if not res:
    print("Get overview message error")
    free_lbmanager(lbm)
print("LB id: ", res[0].lb_id)
assert(res[0].name == "mylb")
assert(res[0].lb_status.senderAddressList == ip_list)
print(len(res[0].lb_status.workerStatusList))

Create LBManager obj succeeded:  0
Register worker succeeded

Session id is:  4
Session token is:  8800e7562f0a9a4b78bff20d8a20a9047201faaee9658c65e41e57b8e3fe41af

Send state for 25 times

Get LB status succeeded:  <e2sar_py.ControlPlane.LoadBalancerStatusReply object at 0x7f9a0e4391f0>
Sender addresses: ['192.168.20.1', '192.168.20.2']
Current Epoch: 2
Current predicted Event No: 9223372036854775808
Workers: [<e2sar_py.ControlPlane.WorkerStatus object at 0x7f9a0e5ee1f0>]
Timestamp: 2024-10-02T04:18:11.402769466Z
expiresAt: 2024-10-02T05:18:08Z

Send state for 25 times

LB id:  7
1


In [38]:
free_lbmanager(lbm)

Free LB succeeded!
